In [3]:
pip install pandas


  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.6 MB 11.1 MB/s eta 0:00:01
   --- ------------------------------------ 1.1/11.6 MB 13.4 MB/s eta 0:00:01
   ----- ---------------------------------- 1.6/11.6 MB 12.9 MB/s eta 0:00:01
   ------- -------------------------------- 2.2/11.6 MB 12.4 MB/s eta 0:00:01
   --------- ------------------------------ 2.7/11.6 MB 12.3 MB/s eta 0:00:01
   ----------- ---------------------------- 3.2/11.6 MB 12.1 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.6 MB 12.0 MB/s eta 0:00:01
   -------------- ------------------------- 4.3/11.6 MB 12.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.9/11.6 MB 12.0 MB/s eta 0:00:01
   ------------------ --

In [4]:
import psycopg2
import pandas as pd

In [5]:
#connection
conn = psycopg2.connect(
    dbname = 'crypto_db',
    user = 'paxdefi',
    password = 'paxDeFi',
    host = '34.42.102.163',
    port = '5432'
)

In [6]:
# Create a cursor object
cur = conn.cursor()

In [7]:
#select all rows of table
#cur.execute('SELECT * from ohlc_daily')
#cur.execute('SELECT * FROM ohlc_daily AS A WHERE A."Symbol" LIKE \'BTCUSDT\'')
symbol = 'BTCUSDT'  # or any other symbol you want to search for
query = 'SELECT * FROM ohlc_daily AS A WHERE A."Symbol" LIKE %s'
cur.execute(query, (symbol,))
 # Fetch the first 10 rows
rows = cur.fetchall()

In [8]:
#rollback if error
conn.rollback()

In [9]:
df = pd.DataFrame(rows)
columns = ['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume', 'Trade Count']
df.columns = columns
df.head()

,Date,Symbol,Open,High,Low,Close,Volume,Trade Count
0,2024-04-13,BTCUSDT,67116.52,67929.00,60660.57,63924.51,4.614469e+09,3341340
1,2024-04-12,BTCUSDT,70006.22,71227.46,65086.86,67116.52,3.843552e+09,2496617
2,2024-04-11,BTCUSDT,70631.08,71305.89,69567.21,70006.23,2.247473e+09,1658995
3,2024-04-10,BTCUSDT,69146.00,71172.08,67518.00,70631.08,2.901565e+09,1852170
4,2024-04-09,BTCUSDT,71620.00,71758.19,68210.00,69146.00,2.749007e+09,1804157


In [10]:
#get rows where column "Symbol" is 'BTCUSDT' 
#symbol = 'BTCUSDT'
symbol_df = df[df['Symbol'].str.contains(symbol)]

#make day of week column from the dates
symbol_df['Date'] = pd.to_datetime(symbol_df['Date'])
#make week column from the dates
symbol_df['week'] = symbol_df['Date'].dt.to_period('W')
#make day column from the dates
symbol_df['day']= symbol_df['Date'].dt.day_name()
#symbol_df = symbol_df.drop(columns=['date'], axis = 1)
symbol_df = symbol_df.set_index('Date')
symbol_df.head()

,Symbol,Open,High,Low,Close,Volume,Trade Count,week,day
Date,,,,,,,,,
2024-04-13,BTCUSDT,67116.52,67929.00,60660.57,63924.51,4.614469e+09,3341340,2024-04-08/2024-04-14,Saturday
2024-04-12,BTCUSDT,70006.22,71227.46,65086.86,67116.52,3.843552e+09,2496617,2024-04-08/2024-04-14,Friday
2024-04-11,BTCUSDT,70631.08,71305.89,69567.21,70006.23,2.247473e+09,1658995,2024-04-08/2024-04-14,Thursday
2024-04-10,BTCUSDT,69146.00,71172.08,67518.00,70631.08,2.901565e+09,1852170,2024-04-08/2024-04-14,Wednesday
2024-04-09,BTCUSDT,71620.00,71758.19,68210.00,69146.00,2.749007e+09,1804157,2024-04-08/2024-04-14,Tuesday


In [11]:
#clean into just date, ohlc,
ohlc_df = symbol_df[['Open', 'High', 'Low', 'Close']]
ohlc_df.head()

,Open,High,Low,Close
Date,,,,
2024-04-13,67116.52,67929.00,60660.57,63924.51
2024-04-12,70006.22,71227.46,65086.86,67116.52
2024-04-11,70631.08,71305.89,69567.21,70006.23
2024-04-10,69146.00,71172.08,67518.00,70631.08
2024-04-09,71620.00,71758.19,68210.00,69146.00


In [12]:
#find return for each individual month
monthly_open_close = ohlc_df.resample('M').agg({'Open': 'first', 'Close': 'last'})
monthly_open_close['monthly return (%)'] = (((monthly_open_close['Close'] - monthly_open_close['Open']) / monthly_open_close['Open']) *100).round(2)
monthly_open_close.head()

C:\Users\myrcene\AppData\Local\Temp\ipykernel_8860\3753641492.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_open_close = ohlc_df.resample('M').agg({'Open': 'first', 'Close': 'last'})


,Open,Close,monthly return (%)
Date,,,
2017-08-31,4261.48,4724.89,10.87
2017-09-30,4689.89,4378.51,-6.64
2017-10-31,4378.49,6463.00,47.61
2017-11-30,6463.00,9838.96,52.24
2017-12-31,9837.00,13716.36,39.44


In [13]:
# Group by name of month and calculate average monthly return
average_monthly_returns = monthly_open_close.groupby(monthly_open_close.index.month).agg({'monthly return (%)': 'mean'})
#change month to corresponding month name
# Find the month(s) with the highest average return
average_monthly_returns.index = pd.to_datetime(average_monthly_returns.index, format='%m').strftime('%B')
average_monthly_returns = average_monthly_returns.reset_index()
average_monthly_returns.columns = ['month', 'average monthly return (%)']
sorted_monthly_returns = average_monthly_returns.sort_values(by='average monthly return (%)', ascending=False)
sorted_monthly_returns

,month,average monthly return (%)
9,October,22.265714
1,February,13.547143
6,July,11.486667
3,April,10.141429
11,December,8.992857
0,January,5.130000
10,November,3.797143
2,March,3.581429
4,May,-1.088333
7,August,-1.744286


In [14]:
#find return for each individual day
daily_open_close = ohlc_df.groupby(pd.Grouper(freq='D')).agg({'Open': 'first', 'Close': 'last'})
daily_open_close['daily return (%)'] = (((daily_open_close['Close'] - daily_open_close['Open']) / daily_open_close['Open']) *100).round(2)
# Assuming df is your DataFrame with datetime index
#Add column of day name
daily_open_close['day']= daily_open_close.index.day_name()
daily_open_close['week'] = daily_open_close.index.to_period('W')
daily_open_close.head()

,Open,Close,daily return (%),day,week
Date,,,,,
2017-08-17,4261.48,4285.08,0.55,Thursday,2017-08-14/2017-08-20
2017-08-18,4285.08,4108.37,-4.12,Friday,2017-08-14/2017-08-20
2017-08-19,4108.37,4139.98,0.77,Saturday,2017-08-14/2017-08-20
2017-08-20,4120.98,4086.29,-0.84,Sunday,2017-08-14/2017-08-20
2017-08-21,4069.13,4016.00,-1.31,Monday,2017-08-21/2017-08-27


In [15]:
# Group by name of day and calculate average daiy return
average_daily_returns = daily_open_close.groupby('day').agg({'daily return (%)': 'mean'})
sorted_avg_daily_returns = average_daily_returns.sort_values(by='daily return (%)', ascending=False)
sorted_avg_daily_returns

,daily return (%)
day,
Wednesday,0.471009
Monday,0.452248
Friday,0.288851
Saturday,0.181437
Sunday,0.079539
Tuesday,0.054092
Thursday,-0.219914


In [16]:
# Group by the 'week' column
grouped_weekly = symbol_df.groupby('week')

# Initialize dictionaries to store counts of high and low occurrences for each day
high_counts = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 0, 'Sunday': 0}
low_counts = {'Monday': 0, 'Tuesday': 0, 'Wednesday': 0, 'Thursday': 0, 'Friday': 0, 'Saturday': 0, 'Sunday': 0}

# Iterate over each group
for week, group_df in grouped_weekly:
    # Find the index of the maximum and minimum values in the 'high' and 'low' columns respectively
    max_high_index = group_df['High'].idxmax()
    min_low_index = group_df['Low'].idxmin()
    
    # Get the day of the week corresponding to the maximum and minimum values
    max_high_day = group_df.loc[max_high_index, 'day']
    min_low_day = group_df.loc[min_low_index, 'day']
    
    # Increment the count for the corresponding day of the week
    high_counts[max_high_day] += 1
    low_counts[min_low_day] += 1

# Sort the high and low counts dictionaries in descending order
sorted_high_counts = sorted(high_counts.items(), key=lambda x: x[1], reverse=True)
sorted_low_counts = sorted(low_counts.items(), key=lambda x: x[1], reverse=True)

# Create DataFrames from the sorted counts dictionaries
high_df = pd.DataFrame(sorted_high_counts, columns=['Day', 'High_Count'])
low_df = pd.DataFrame(sorted_low_counts, columns=['Day', 'Low_Count'])

#create variables for the most common high and low of the week
most_common_high_of_week = high_df.iloc[0]['Day']
most_common_low_of_week = low_df.iloc[0]['Day']

# Print the DataFrames
print("Most commonly the high of the week occurs on: " + most_common_high_of_week)
print("\nMost commonly the low of the week occurs on: " + most_common_low_of_week)
#display the data
display(high_df, low_df)

Most commonly the high of the week occurs on: Monday

Most commonly the low of the week occurs on: Monday


,Day,High_Count
0,Monday,92
1,Sunday,58
2,Tuesday,45
3,Wednesday,45
4,Saturday,42
5,Thursday,34
6,Friday,32


,Day,Low_Count
0,Monday,120
1,Friday,47
2,Tuesday,43
3,Thursday,42
4,Wednesday,39
5,Sunday,39
6,Saturday,18


In [18]:
# Assuming df is your DataFrame with OHLC data and the index is a DatetimeIndex

# Calculate the difference between current day's high and prior day's high
ohlc_df['high_difference'] = ohlc_df['High'] - ohlc_df['High'].shift(1)

# Calculate the difference between current day's low and prior day's low
ohlc_df['low_difference'] = ohlc_df['Low'] - ohlc_df['Low'].shift(1)

# Count the number of times both differences are positive
both_positive_count = ((ohlc_df['high_difference'] > 0) & (ohlc_df['low_difference'] < 0)).sum()

# Total number of data points
total_data_points = len(ohlc_df)

# Calculate the percentage
percentage = round(((both_positive_count / total_data_points) * 100),2)

print('The amount of times we run both the prior day high and low is: ' + str(percentage) + '%')


The amount of times we run both the prior day high and low is: 21.38%


C:\Users\myrcene\AppData\Local\Temp\ipykernel_8860\1673698711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ohlc_df['high_difference'] = ohlc_df['High'] - ohlc_df['High'].shift(1)
C:\Users\myrcene\AppData\Local\Temp\ipykernel_8860\1673698711.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ohlc_df['low_difference'] = ohlc_df['Low'] - ohlc_df['Low'].shift(1)


Below does not seem accurate, but its supposed to give the % of days that we take PWH / PWL

In [19]:
# Assuming df is your DataFrame with daily OHLC data and the index is a DatetimeIndex
# Resample the daily data into weekly data
weekly_df = ohlc_df.resample('W').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
# Assuming df is your DataFrame

# Shift the weekly data by one week to get the previous week's data
previous_week_df = weekly_df.shift(-1)

# Calculate the difference between current week's high and previous week's high
weekly_df['high_difference'] = weekly_df['High'] - previous_week_df['High']

# Calculate the difference between current week's low and previous week's low
weekly_df['low_difference'] = weekly_df['Low'] - previous_week_df['Low']

# Count the number of times the high is positive and the low is negative
both_positive_count = ((weekly_df['high_difference'] > 0) & (weekly_df['low_difference'] < 0)).sum()

# Total number of data points
total_data_points = len(weekly_df)

# Calculate the percentage
percentage = (both_positive_count / total_data_points) * 100
# Assuming df is your DataFrame
weekly_df = weekly_df.drop(weekly_df.index[0])

print(f"Probability of price taking out PWH / PWL in same week: {percentage:.2f}%")
weekly_df.head()


Probability of price taking out PWH / PWL in same week: 15.23%


,Open,High,Low,Close,high_difference,low_difference
Date,,,,,,
2017-08-27,4069.13,4453.91,3400.00,4310.01,-485.28,-724.54
2017-09-03,4310.01,4939.19,4124.54,4509.08,150.60,521.54
2017-09-10,4505.00,4788.59,3603.00,4130.37,394.00,786.00
2017-09-17,4153.62,4394.59,2817.00,3699.99,271.39,-688.55
2017-09-24,3690.00,4123.20,3505.55,3660.02,-283.32,-148.14
